In [8]:
import re
import pandas as pd
from konlpy.tag import Okt
import json
from moviepy import VideoFileClip, CompositeVideoClip
from moviepy.video.VideoClip import ImageClip
import numpy as np
from PIL import Image

In [9]:
# 검색 키워드들(== 등록된 상품명들)
# TODO 추후 redis로 옮기는 것 고려
registered_goods = [
    '가죽 자켓', '청자켓', '후드', '맥코트', '신발', '바지', '선글라스', '(메시)니트', '반팔티', '이너', '아우터', 
    '데님 자켓', '데님 팬츠', '데님 트러커', '자켓', '워크자켓', '아디다스 저지', '아디다스 트랙탑', '스카프', '토니웩', '모모타로진'
]

# Okt 객체 생성
okt = Okt()
df = pd.read_csv('../../data/csv/36dc8bba-c0ae-475c-8402-04a1d716b647.csv', header=None, names=["id", "start", "end", "text"])

df.head()

,id,start,end,text
0,b01354b8-2932-4256-8292-fc0b54824a90,00:00:00,00:00:02,잘 입을 수 있는 그런 자켓인 것 같고
1,b80da18b-3bd4-4126-862c-8e61f8292f83,00:00:02,00:00:03,계절 걱정 많이 하셨잖아요
2,92144837-f1b3-47c5-9cd5-fef220b83400,00:00:03,00:00:08,가죽 자켓이나 특히 청자켓 가죽 자켓은 잠깐 입고


In [10]:
# TODO 중복으로 등장하는 키워드가 있는 경우 처음 시간대만 체크하는 것 고려
# TODO 해당 코드 이해
normalized_patterns = [
    re.escape(item).replace(r"\ ", r"\s*") for item in registered_goods # 공백이 있든 없든 탐지(가죽 자켓 / 가죽자켓)
]

pattern = re.compile("|".join(normalized_patterns))

# 매칭된 키워드 리스트를 새로운 컬럼으로 추가
df["matched_keywords"] = (
    df["text"]
    .str.findall(pattern)
    .map(set)   # set으로 변환 → 중복 제거
    .map(list)  
)

df

,id,start,end,text,matched_keywords
0,b01354b8-2932-4256-8292-fc0b54824a90,00:00:00,00:00:02,잘 입을 수 있는 그런 자켓인 것 같고,[자켓]
1,b80da18b-3bd4-4126-862c-8e61f8292f83,00:00:02,00:00:03,계절 걱정 많이 하셨잖아요,[]
2,92144837-f1b3-47c5-9cd5-fef220b83400,00:00:03,00:00:08,가죽 자켓이나 특히 청자켓 가죽 자켓은 잠깐 입고,"[청자켓, 가죽 자켓]"


In [11]:
# 매칭된 게 있는 row만 추출
matched_df = df[df["matched_keywords"].str.len() > 0]

print(matched_df)

                                     id      start        end  \
0  b01354b8-2932-4256-8292-fc0b54824a90   00:00:00   00:00:02   
2  92144837-f1b3-47c5-9cd5-fef220b83400   00:00:03   00:00:08   

                           text matched_keywords  
0         잘 입을 수 있는 그런 자켓인 것 같고             [자켓]  
2   가죽 자켓이나 특히 청자켓 가죽 자켓은 잠깐 입고     [청자켓, 가죽 자켓]  


In [12]:
def resize_array(img: np.ndarray, width=None, height=None):
    pil_img = Image.fromarray(img)
    if width and height:
        new_img = pil_img.resize((width, height))
    elif width:
        # 비율 유지
        h = int(pil_img.height * width / pil_img.width)
        new_img = pil_img.resize((width, h))
    elif height:
        w = int(pil_img.width * height / pil_img.height)
        new_img = pil_img.resize((w, height))
    else:
        new_img = pil_img
    return np.array(new_img)

In [ ]:
with open('../../data/json/fashion.json', 'r', encoding='utf-8') as f:
    keyword_map = json.load(f)
    
for _, row in matched_df.iterrows():
    for keyword in row['matched_keywords']:
        if keyword in keyword_map.keys():
            goods_info = keyword_map[keyword]
            
            video = VideoFileClip("../../data/video/36dc8bba-c0ae-475c-8402-04a1d716b647.mp4")

            # TODO 상품 이미지가 나오는 공간을 YOLO나 SAM으로 파악하여 빈공간에 나올 수 있도록 수정
            img = (
                ImageClip(goods_info['image_path'], duration=5)           
                    .with_start(3)
                    .with_position((0.7, 0.05), relative=True)
            )

            # 영상 + 이미지 합성
            final = CompositeVideoClip([video, img])

            # 저장
            final.write_videofile("output.mp4", codec="libx264", audio_codec="aac")
            
            print(row)
            print(goods_info)

MoviePy - Building video output.mp4.
MoviePy - Writing audio in outputTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
MoviePy - Writing video output.mp4



MoviePy - Done !
MoviePy - video ready output.mp4
id                  92144837-f1b3-47c5-9cd5-fef220b83400
start                                           00:00:03
end                                             00:00:08
text                         가죽 자켓이나 특히 청자켓 가죽 자켓은 잠깐 입고
matched_keywords                            [청자켓, 가죽 자켓]
Name: 2, dtype: object
{'id': '57eea140-71dc-4915-b8f2-b617fdbf40d9', 'image_path': '../../data/image/leather_jacket_total_3.jpg', 'link': 'https://noirlarmes.com/m-leather/?idx=1209&srsltid=AfmBOoprDztmtZhGrxB1T1wK0qTy1rKL9sZ5mnheE0i3xi7F6eb06yqJ'}
